<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: Maria Paz Raveau</em><br>
    <em>Asignatura: Procesamiento de Lenguaje Natural</em><br>

</div>

# **Tarea 1**

*Fecha de Entrega: Domingo 20, Julio 2025.*

**Nombre Estudiante**: Victor Saldivia Vera

### **01. Intrucciones y Enunciado**

Suponga que ud. es contactado por alguna agencia gubernamental que quiere saber qué dijeron los participantes de los cabildos de 2016. Ud. debe entonces, basándose en los textos de fundamento, responder a este requerimiento.  Use para ello la base de datos que vimos en clases, de los cabildos provinciales. 

- Escoja alguna de los conceptos que vienen en la base de datos (evite las que no tengan muchas entradas). Hemos visto cómo generar WordClouds con tokens y con bigramas. *¿Es posible valerse de las etiquetas gramaticales para mejorar el resultado?* 
- Pruebe haciendo un WordCloud con tokens/sustantivos,  y otro con  bigramas/sustantivo-adjetivo. Recuerde pre-procesar correctamente.
- Con estos mismo bigramas/sustantivo-adjetivo, haga una red de bigramas.
- Ayúdese con frases de ejemplo, y de los WordClouds, para escribir una descripción del concepto.


### **02. Descripción de la Data**

Carga y primeras exploraciones del DataFrame original.  
- Columnas disponibles: 12  
- Tamaño del dataset: 30414 registros  
- Conteo de nulos en las columnas de texto: 0

In [105]:
import pandas as pd
df = pd.read_excel('data/resultadocabildoprovincial.xlsx')
print(f'Dimensión: {df.shape}'), 
print(f'Nombre de Columnas: {df.columns}')

Dimensión: (30414, 12)
Nombre de Columnas: Index(['region_nombre', 'provincia_nombre', 'comuna_nombre', 'cabildo_id',
       'cabildo_nombre', 'cabildo_direccion', 'num_mesa', 'pregunta_glosa',
       'concepto_nombre', 'concepto_otro', 'tipo-eleccion', 'fundamento'],
      dtype='object')


### **03. Librerías a Utilizar**
Importación de librerías necesarias:

- **Pandas**, **NumPy** y expresiones regulares para limpieza  
- **Stanza** para NLP (tokenización, POS, lematización)  
- **NLTK** para stopwords y collocations  
- **WordCloud**, **NetworkX** y **Matplotlib** para visualización  
- **tqdm** para barras de progreso.

In [106]:
import numpy as np
import re
import os
import unicodedata
from collections import Counter

# Librerías de Procesamiento de Texto
import nltk
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import stanza

# Visualización
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from wordcloud import WordCloud
import networkx as nx

# La librería de warnings se utiliza para manejar las advertencias
import warnings
warnings.filterwarnings('ignore')

# Paquete muestra Barra de Progreso
from tqdm import tqdm
tqdm.pandas()

Se estableció `KMP_DUPLICATE_LIB_OK=TRUE` para evitar conflicto de múltiples runtimes OpenMP en Windows. Es un workaround temporal *(solución temporal)*.

In [107]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
# Descargas (Nota: Ejecutar una sola vez en entorno local)
# nltk.download('punkt')
# nltk.download('stopwords')
# stanza.download('es')

#### Configuración de Stanza para el NLP

In [108]:
nlp = stanza.Pipeline('es', processors='tokenize,pos,lemma', use_gpu=False)  # Cambiar a True si hay GPU


2025-07-20 18:43:36 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-20 18:43:36 INFO: Downloaded file to C:\Users\Victo\stanza_resources\resources.json
2025-07-20 18:43:36 WARNING: Language es package default expects mwt, which has been added
2025-07-20 18:43:37 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2025-07-20 18:43:37 INFO: Using device: cpu
2025-07-20 18:43:37 INFO: Loading: tokenize
2025-07-20 18:43:37 INFO: Loading: mwt
2025-07-20 18:43:37 INFO: Loading: pos
2025-07-20 18:43:39 INFO: Loading: lemma
2025-07-20 18:43:40 INFO: Done loading processors!


### **04. Carga y Exploración Inicial**


In [109]:
df.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...
3,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Equidad de género,NaN,A,Sin fundamento
4,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Sin fundamento


#### Conteo de nulos en columnas de texto


In [110]:
texto_col = "fundamento"
print(df[texto_col].isna().sum())

0


#### Estadísticas Descriptivas sobre el texto


In [111]:
df['len_chars'] = df[texto_col].str.len() # columna de longitud de caracteres
df['len_tokens_raw'] = df[texto_col].str.split().str.len() # columna de tokens sin procesar
df[['len_chars','len_tokens_raw']].describe()

,len_chars,len_tokens_raw
count,30414.000000,30414.000000
mean,69.090123,10.828270
std,69.670394,10.993367
min,1.000000,1.000000
25%,15.000000,2.000000
50%,47.000000,7.000000
75%,102.000000,16.000000
max,300.000000,61.000000


#### Eliminación de nulos y duplicados


In [112]:
before = len(df)
df = df.dropna(subset=[texto_col])
df = df.drop_duplicates(subset=[texto_col])
print(f"Filas eliminadas: {before - len(df)}")

Filas eliminadas: 11088


### **05. Selección de Concepto**
Se exploran las frecuencias de `concepto_nombre` y se elige el concepto.

- Se elige el concepto `Democracia` ya que tienen una alta frecuencia (Top 3) 
- Se construye yb sub conjunto `subset` filtrado por el concepto elegido

In [113]:
# Distribución de frecuencias para elegir uno con suficientes observaciones
concepto_col = "concepto_nombre" 
concept_counts = df[concepto_col].value_counts()
concept_counts.head(20)

concepto_nombre
Otro                                                      3976
Igualdad                                                   519
Democracia                                                 507
Descentralización                                          503
Justicia                                                   492
Respeto/conservación de la naturaleza o medio ambiente     484
A la vida                                                  440
A la salud                                                 437
A la educación                                             428
Respeto                                                    385
Fuerzas Armadas                                            381
Bien Común/comunidad                                       381
Congreso o parlamento (estructura y funciones)             373
A la vivienda digna                                        365
Plebiscitos, referendos y consultas                        361
Respeto por la constitución            

#### Selección de Concepto

In [ ]:
concepto_elegido = "Democracia" 

#### Creación del Subconjunto filtrado
Se elige el concepto 'Democracia' el cual tiene una frecuencia de (507 ocurrencias) para garantizar masa textual suficiente y diversidad léxica. Se descartaron conceptos con baja frecuencia para evitar resultados inestables en nubes de palabras y bigramas.”



In [ ]:
subset = df[df[concepto_col] == concepto_elegido].copy()
print("Filas del concepto:", len(subset))


In [ ]:
# Subconjunto de datos para el concepto elegido
subset.head(10)

### **06. Pre-Procesamiento de Texto**


#### Normalización de Texto

In [ ]:
def normalize_text(text: str) -> str:
    text = str(text).lower() #
    text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8','ignore')  # eliminar tildes
    text = re.sub(r'[^a-z0-9\\s]', ' ', text) # eliminar caracteres especiales
    text = re.sub(r'\\s+', ' ', text).strip() # eliminar espacios extra
    return text

subset['texto_norm'] = subset[texto_col].apply(normalize_text)

In [ ]:
subset

#### Tokenización + POS + Lematización

In [ ]:
def stanza_process(doc_str: str):
    doc = nlp(doc_str)
    rows = []  # (token, lemma, upos)
    for sent in doc.sentences:
        for w in sent.words:
            rows.append((w.text, w.lemma.lower(), w.upos))
    return rows

subset['doc'] = subset['texto_norm'].progress_apply(stanza_process)

#### Visualización de tokens crudos (antes de filtrar stopwords) del subconjunto

In [ ]:
subset['tokens_crudos'] = subset['doc'].apply(lambda toks: [lemma for _, lemma, pos in toks])

freq_crudas = Counter([w for fila in subset['tokens_crudos'] for w in fila])
top30 = freq_crudas.most_common(30)
top30

In [ ]:
subset['doc']

#### Stopwords

In [ ]:
spanish_sw = set(stopwords.words('spanish'))
#extra_stop = {'chile','pais','region','cabildo','provincia'}
#stopwords_total = spanish_sw.union(extra_stop)
spanish_sw

#### Filtrar Sustantivos

In [ ]:
subset['nouns'] = subset['doc'].apply(
    lambda toks: [lemma for _, lemma, pos in toks
                  if pos in {"NOUN","PROPN"} and lemma not in spanish_sw and len(lemma) > 2]
)

In [ ]:
subset['nouns']

#### Filtrar Adjetivos

In [ ]:

subset['adjs'] = subset['doc'].apply(
    lambda toks: [lemma for _, lemma, pos in toks
                  if pos == 'ADJ' and len(lemma) > 2]
)

In [ ]:
subset

#### Bigramas Sustantivo–Adjetivo contiguos


In [ ]:
def noun_adj_bigrams(tok_rows):
    bigs = []
    for i in range(len(tok_rows)-1):
        (_, lemma1, pos1) = tok_rows[i]
        (_, lemma2, pos2) = tok_rows[i+1]
        if pos1 in {'NOUN','PROPN'} and pos2 == 'ADJ':
            if lemma1 not in spanish_sw and lemma2 not in spanish_sw:
                bigs.append(f'{lemma1}_{lemma2}')
    return bigs

subset['bigrams_noun_adj'] = subset['doc'].apply(noun_adj_bigrams)

#### Conteos Globales (Unigramas y Bigramas)

In [ ]:
all_nouns = Counter([w for fila in subset['nouns'] for w in fila])
all_bigrams = Counter([b for fila in subset['bigrams_noun_adj'] for b in fila])
adj_counts = Counter([w for fila in subset['adjs'] for w in fila])
print('Unigramas distintos:', len(all_nouns))
print('Bigramas N-A distintos:', len(all_bigrams))
print('Adjetivos distintos:', len(adj_counts))

#### Remover claves vacías o sospechosas
Se crea la función `clean_key(k)` para verificar que una palabra no esté vacía, y sea de tipo string. También verifica que no contenga espacios en blancos el string. Se crea un diccionario llamado `filtered_items`, donde se agregan las palabras que pasan el filtro de la función `clean_key(k)` y que aparecerán al menos dos veces en el conteo global de sustantivos.

In [ ]:
def clean_key(k):
    return bool(k) and isinstance(k, str) and not k.isspace()

filtered_items = {k:int(v) for k,v in all_nouns.items() if clean_key(k) and v>=2}

print("Vocab filtrado:", len(filtered_items), "Total frecuencia:", sum(filtered_items.values()))

### **07. Visualizaciones**

#### WordCloud Sustantivos - Concepto Elegido

In [ ]:
freq_unigrams = {w:c for w,c in all_nouns.items() if c>=2}
wc = WordCloud(width=1500, height=800, background_color='white', colormap="magma",  max_words=1000, collocations=False)
wc.generate_from_frequencies(freq_unigrams)
plt.figure(figsize=(12,8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title(f'WordCloud Sustantivos – {concepto_elegido}')
plt.show()

#### WordCloud Bigramas (Sustantivo-Adjetivo)

In [ ]:
freq_bigrams = {b:c for b,c in all_bigrams.items() if c>=2}
freq_bigrams_space = {b.replace('_', ' '): c for b, c in freq_bigrams.items()} # Se crea un nuevo diccionario reemplazando '_' por ' '

wc2 = WordCloud(
    width=1500,
    height=800,
    background_color='white',
    colormap="magma",
    max_words=400,
    collocations=False
).generate_from_frequencies(freq_bigrams_space)

plt.figure(figsize=(12,8))
plt.imshow(wc2, interpolation='bilinear')
plt.axis('off')
plt.title(f'WordCloud Bigramas Sustantivo - Adjetivo – {concepto_elegido}')
plt.show()

#### Top 20 Adjetivos - Democracia

In [ ]:
# Nuevo dataFrame de adjetivos
adj_df = pd.DataFrame(adj_counts.most_common(20), columns=['adjetivo','frecuencia'])

plt.figure(figsize=(10,6))
bars = plt.barh(adj_df['adjetivo'], adj_df['frecuencia'], color="#ED7272")

# Etiquetas de datos
for bar in bars:
    width = bar.get_width()
    plt.text(
        width + max(adj_df['frecuencia']) * 0.01, 
        bar.get_y() + bar.get_height()/2,
        f'{int(width)}',
        va='center',
        fontsize=9
    )

plt.xlabel('Frecuencia')
plt.ylabel('Adjetivo')
plt.title(f'Top 20 Adjetivos – {concepto_elegido}')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

#### Red de Bigramas

In [ ]:
G = nx.Graph()
for big, freq in freq_bigrams.items():
    noun, adj = big.split('_')
    G.add_node(noun, tipo='NOUN')
    G.add_node(adj,  tipo='ADJ')
    G.add_edge(noun, adj, weight=freq)

print('Nodos:', G.number_of_nodes(), '| Aristas:', G.number_of_edges())

# Cálculo de métricas para tamaño y color
degree_dict = dict(G.degree())

# Se escala el tamaño de nodo según grado
node_sizes  = [100 + degree_dict[n]*200 for n in G.nodes()]
color_map = {'NOUN':"#F88A8A", 'ADJ':"#FFC18B"}
node_colors = [color_map[G.nodes[n]['tipo']] for n in G.nodes()]

# Se escala la anchura de arista según peso 
edge_widths = [G[u][v]['weight']*0.1 for u,v in G.edges()]

plt.figure(figsize=(16,14))
pos = nx.spring_layout(G, k=0.5, seed=42)

nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.6)
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.9)
nx.draw_networkx_labels(G, pos, font_size=9)

legend_elems = [
    Patch(facecolor=color_map['NOUN'], label='Sustantivo'),
    Patch(facecolor=color_map['ADJ'],  label='Adjetivo')
]
plt.legend(handles=legend_elems, title='Tipo Nodo')

plt.title(f'Red de Bigramas Sustantivo–Adjetivo – {concepto_elegido}')
plt.axis('off')
plt.tight_layout()
plt.show()